### 1. Setup and Import Libraries


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Example user and video data
user_data = {
    'user_id': [1, 2, 3],
    'watch_history': [
        ['Tech Review', 'AI in Healthcare', 'Python Tutorial'],
        ['Travel Vlog', 'Food Recipe', 'Cultural Documentary'],
        ['AI in Healthcare', 'Deep Learning Lecture', 'Tech Review']
    ],
    'language': ['English', 'Spanish', 'English'],
    'location': ['USA', 'Spain', 'UK']
}

video_data = {
    'video_id': [101, 102, 103, 104, 105, 106],
    'title': ['Tech Review', 'AI in Healthcare', 'Python Tutorial', 'Travel Vlog', 'Food Recipe', 'Cultural Documentary'],
    'category': ['Technology', 'Health', 'Education', 'Travel', 'Food', 'Documentary'],
    'rating': [4.5, 4.8, 4.7, 4.2, 4.3, 4.6],
    'view_counts': [12000, 18000, 15000, 10000, 11000, 13000],
    'share_counts': [300, 500, 400, 200, 250, 300]
}

In [ ]:
# Converting data to DataFrames
users_df = pd.DataFrame(user_data)
videos_df = pd.DataFrame(video_data)

### 2. Content-Based Filtering


In [ ]:
# Create a TF-IDF Vectorizer for video titles
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# Fit and transform the video titles
tfidf_matrix = tfidf_vectorizer.fit_transform(videos_df['title'])

In [ ]:
# Compute the cosine similarity between all videos
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Function to recommend videos based on content similarity
def recommend_videos(user_id, user_watch_history, n_recommendations=3):
    watched_video_indices = []
    for video_title in user_watch_history:
        index = videos_df[videos_df['title'] == video_title].index[0]
        watched_video_indices.append(index)
    
    similarity_scores = sum(cosine_sim_matrix[watched_video_indices])
    similar_videos_indices = similarity_scores.argsort()[-(n_recommendations+len(user_watch_history)):-len(user_watch_history)]
    
    recommended_videos = videos_df.iloc[similar_videos_indices][['title', 'category', 'rating']]
    return recommended_videos

In [ ]:
# Example: Get recommendations for user 1
user_id = 1
user_watch_history = users_df.loc[users_df['user_id'] == user_id, 'watch_history'].values[0]
recommendations = recommend_videos(user_id, user_watch_history)
print("Recommended Videos for User 1:")
print(recommendations)

### 3. Collaborative Filtering

In [ ]:
# Function to recommend videos based on what similar users have watched
def collaborative_filtering(user_id, users_df, n_recommendations=3):
    user_watch_history = users_df.loc[users_df['user_id'] == user_id, 'watch_history'].values[0]
    
    other_users = users_df[users_df['user_id'] != user_id]
    similar_users = other_users[other_users['language'] == users_df.loc[users_df['user_id'] == user_id, 'language'].values[0]]
    
    recommendations = []
    for index, row in similar_users.iterrows():
        for video in row['watch_history']:
            if video not in user_watch_history and video not in recommendations:
                recommendations.append(video)
    
    recommended_videos = videos_df[videos_df['title'].isin(recommendations)].head(n_recommendations)
    return recommended_videos

In [ ]:
# Example: Get recommendations for user 1 using collaborative filtering
collab_recommendations = collaborative_filtering(user_id, users_df)
print("Collaborative Recommendations for User 1:")
print(collab_recommendations)

### 4. Hybrid Approach

In [ ]:
# Function to combine content-based and collaborative filtering
def hybrid_recommendation(user_id, users_df, videos_df, n_recommendations=5):
    content_based_recommendations = recommend_videos(user_id, users_df.loc[users_df['user_id'] == user_id, 'watch_history'].values[0])
    collaborative_recommendations = collaborative_filtering(user_id, users_df)
    
    hybrid_recommendations = pd.concat([content_based_recommendations, collaborative_recommendations]).drop_duplicates().head(n_recommendations)
    return hybrid_recommendations

In [ ]:
# Example: Get hybrid recommendations for user 1
hybrid_recommendations = hybrid_recommendation(user_id, users_df, videos_df)
print("Hybrid Recommendations for User 1:")
print(hybrid_recommendations)